In [48]:
import pymc as pm
import numpy as np
import pandas as pd
import re




In [49]:
df = pd.read_csv('../data/modeling_data.csv')
pd.set_option('display.max_columns', None)
df = df.fillna(0)
df.head()

C:\Users\ksbuf\AppData\Local\Temp\ipykernel_7532\2417960917.py:1: DtypeWarning: Columns (208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/modeling_data.csv')


,student_number,ac_ind,overall_gpa,days_attended,days_absent,school_membership,percent_days_attended,is_one_percent_y,extended_school_year_y,regular_percent_1.0,regular_percent_2.0,regular_percent_3.0,environment_h,environment_r,environment_v,gender_f,gender_m,gender_u,ethnicity_y,amerindian_alaskan_y,asian_y,black_african_amer_y,hawaiian_pacific_isl_y,white_y,migrant_y,military_child_y,refugee_student_y,gifted_y,services_504_y,immigrant_y,passed_civics_exam_y,reading_intervention_y,home_status_0,home_status_1,home_status_2,home_status_3,home_status_4,home_status_5,hs_complete_status_ao,hs_complete_status_ct,hs_complete_status_do,hs_complete_status_gc,hs_complete_status_gg,hs_complete_status_gq,hs_complete_status_gr,hs_complete_status_rt,hs_complete_status_nan,part_time_home_school_h,part_time_home_school_p,part_time_home_school_s,part_time_home_school_nan,tribal_affiliation_g,tribal_affiliation_n,tribal_affiliation_o,tribal_affiliation_p,tribal_affiliation_u,tribal_affiliation_nan,tribal_affiliation_s,limited_english_f,limited_english_n,limited_english_o,limited_english_y,limited_english_nan,ell_instruction_type_dul,ell_instruction_type_esl,ell_instruction_type_nan,ell_instruction_type_imt,ell_instruction_type_sei,ell_native_language_ace,ell_native_language_afr,ell_native_language_amh,ell_native_language_ara,ell_native_language_bur,ell_native_language_chi,ell_native_language_cmn,ell_native_language_cpe,ell_native_language_cpp,ell_native_language_cus,ell_native_language_dan,ell_native_language_eng,ell_native_language_fre,ell_native_language_ger,ell_native_language_hat,ell_native_language_hin,ell_native_language_ind,ell_native_language_ita,ell_native_language_jpn,ell_native_language_khm,ell_native_language_kor,ell_native_language_lin,ell_native_language_mah,ell_native_language_nav,ell_native_language_nub,ell_native_language_por,ell_native_language_rus,ell_native_language_slv,ell_native_language_smo,ell_native_language_som,ell_native_language_spa,ell_native_language_swe,ell_native_language_tgl,ell_native_language_ton,ell_native_language_tur,ell_native_language_ukr,ell_native_language_vie,ell_native_language_xea,ell_native_language_yue,ell_native_language_zun,ell_native_language_nan,ell_native_language_arm,ell_native_language_fil,ell_native_language_fin,ell_native_language_gre,ell_native_language_heb,ell_native_language_may,ell_native_language_nor,ell_native_language_tha,ell_native_language_ase,ell_native_language_ben,ell_native_language_kan,ell_native_language_kar,ell_native_language_lao,ell_native_language_nep,ell_native_language_pol,ell_native_language_shh,ell_native_language_tam,ell_native_language_prs,ell_native_language_swa,ell_native_language_wol,ell_parent_language_afr,ell_parent_language_amh,ell_parent_language_ara,ell_parent_language_chi,ell_parent_language_cmn,ell_parent_language_eng,ell_parent_language_fre,ell_parent_language_ger,ell_parent_language_hin,ell_parent_language_jpn,ell_parent_language_khm,ell_parent_language_kor,ell_parent_language_mah,ell_parent_language_nav,ell_parent_language_nub,ell_parent_language_por,ell_parent_language_rus,ell_parent_language_smo,ell_parent_language_som,ell_parent_language_spa,ell_parent_language_tgl,ell_parent_language_ton,ell_parent_language_tur,ell_parent_language_vie,ell_parent_language_yue,ell_parent_language_nan,ell_parent_language_bur,ell_parent_language_cpe,ell_parent_language_heb,ell_parent_language_ita,ell_parent_language_may,ell_parent_language_swe,ell_parent_language_tha,ell_parent_language_alb,ell_parent_language_arm,ell_parent_language_ase,ell_parent_language_ben,ell_parent_language_kan,ell_parent_language_lao,ell_parent_language_nep,ell_parent_language_pan,ell_parent_language_pol,ell_parent_language_tam,ell_parent_language_fil,ell_parent_language_prs,ell_parent_language_sin,ell_parent_language_swa,ell_parent_language_ukr,ell_parent_language_wol,ell_parent_language_ace,ell_parent_language_per,read_grade_level_y,exit_code_11,exit_code_de,exit_code_do,exit_code_ex,exit_c

In [50]:
# Create ELL column: 1 if an ELL student , 0 if native English speaker
df["ell_status"] = df[["limited_english_y", "limited_english_o"]].max(axis=1)

#create the disability status column, if any of "regular_percent_1.0", "regular_percent_2.0", "regular_percent_3.0" show one then disability status = 1 else 0
df["disability_status"] = df[["regular_percent_1.0", "regular_percent_2.0", "regular_percent_3.0"]].max(axis=1)


In [51]:
# Create a column for the four groups
df["ell_dis_group"] = (
    df["ell_status"].astype(str) + "_" + df["disability_status"].astype(str)
)

# Map it to meaningful labels
group_mapping = {
    "1_1": "ELL with Disability",
    "1_0": "ELL without Disability",
    "0_1": "Non-ELL with Disability",
    "0_0": "Non-ELL without Disability",
}
df["ell_dis_group"] = df["ell_dis_group"].map(group_mapping)

In [52]:
df["ell_dis_group"].value_counts()


ell_dis_group
Non-ELL without Disability    16320
Non-ELL with Disability        1596
ELL without Disability          412
ELL with Disability             146
Name: count, dtype: int64

In [53]:
#This will assign a unique integer (0, 1, 2, 3) to each of the four groups.

df["ELL_Disability_Index"] = df["ell_dis_group"].astype("category").cat.codes
group_labels = dict(enumerate(df["ell_dis_group"].astype("category").cat.categories))


In [54]:

# Convert categorical group column to an array
group_idx = df["ELL_Disability_Index"].values
num_groups = df["ELL_Disability_Index"].nunique()

# Predictor variables (example)
X = df.drop(columns = ['limited_english_y', 'limited_english_o', "regular_percent_1.0",
                        "regular_percent_2.0", "regular_percent_3.0", 'ell_dis_group', 'ell_status', 
                        'disability_status', 'entry_date', 'test_date']).values
y = df["ac_ind"].values  # Assuming binary outcome

with pm.Model() as model:
    # Group-level intercepts (varying by ELL-Disability group)
    mu_a = pm.Normal("mu_a", mu=0, sigma=1)  # Hyperprior for intercept mean
    sigma_a = pm.HalfNormal("sigma_a", 1)   # Hyperprior for variance
    a = pm.Normal("a", mu=mu_a, sigma=sigma_a, shape=num_groups)  # Varying intercepts

    # Fixed effect coefficients (one per predictor)
    b = pm.Normal("b", mu=0, sigma=1, shape=(X.shape[1],))  # Make `b` a vector

    # Linear model (proper matrix multiplication)
    logit_p = a[group_idx] + pm.math.dot(X, b)  # No more flattening!

    # Sigmoid transformation
    p = pm.Deterministic("p", pm.math.sigmoid(logit_p))

    # Likelihood
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y)

    # Sample
    trace = pm.sample(2000, tune=1000, target_accept=0.9)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_a, sigma_a, a, b]


c:\Users\ksbuf\anaconda3\envs\pymc_env\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 3676 seconds.
There were 8000 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [55]:
model.debug()

point={'mu_a': array(0.), 'sigma_a_log__': array(0.), 'a': array([0., 0., 0., 0.]), 'b': array([0., 0., 0., ..., 0., 0., 0.])}

No problems found


In [56]:

# Summarize results
pm.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_a,-0.167,0.205,-0.481,0.070,0.102,0.078,4.0,4.0,1.865000e+01
a[0],-0.499,0.322,-0.843,-0.102,0.161,0.123,4.0,4.0,5.450000e+00
a[1],0.013,0.548,-0.659,0.783,0.273,0.209,4.0,4.0,1.465000e+01
a[2],-0.414,0.685,-1.035,0.728,0.342,0.262,4.0,4.0,1.789000e+01
a[3],-0.182,0.412,-0.585,0.464,0.205,0.157,4.0,4.0,1.276000e+01
...,...,...,...,...,...,...,...,...,...
p[18469],0.750,0.433,0.000,1.000,0.216,0.166,4.0,4.0,5.617601e+15
p[18470],0.750,0.433,0.000,1.000,0.216,0.166,4.0,4.0,5.617601e+15
p[18471],0.750,0.433,0.000,1.000,0.216,0.166,4.0,4.0,5.617601e+15
p[18472],0.750,0.433,0.000,1.000,0.216,0.166,4.0,4.0,5.617601e+15
